# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. 

## Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.isna().values.any()

False

In [4]:
df.columns[df.duplicated().any()]

array([], shape=(0, 5), dtype=object)

Посмторел данные , на всякий случай проверил на пропуски и дубликаты 

## Разбейте данные на выборки

In [5]:
X, X_valid = train_test_split(df,test_size=0.2,train_size=0.8)
X_train, X_test = train_test_split(X ,test_size = 0.25)

len(X_test), len(X_valid), len(X_train)

(643, 643, 1928)

In [6]:
y_train = X_train['is_ultra']
X_train = X_train.drop('is_ultra', axis=1)

In [7]:
y_valid = X_valid['is_ultra']
X_valid = X_valid.drop('is_ultra', axis=1)

In [8]:
y_test = X_test['is_ultra']
X_test = X_test.drop('is_ultra', axis=1)

Разбил 60 20 20 процентов соответственно 

In [9]:
#Kод ревьюера
!pip install fast_ml
from fast_ml.model_development import train_valid_test_split

X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = 'is_ultra', 
                                                                            train_size=0.6, valid_size=0.2, test_size=0.2)


     |████████████████████████████████| 42 kB 355 kB/s eta 0:00:011


## Исследуйте модели

### Tree classifier

#### подбор параметров

In [10]:
params = {
            'criterion' : ('gini', 'entropy', 'log_loss')
             ,'max_depth' : range(1, 10)
             ,'min_samples_leaf' : range(1, 7)
             }

cls_tree = GridSearchCV(
            DecisionTreeClassifier(random_state=1234)
            ,params
            ,cv=3
            ,refit=True)

cls_tree.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=1234),
             param_grid={'criterion': ('gini', 'entropy', 'log_loss'),
                         'max_depth': range(1, 10),
                         'min_samples_leaf': range(1, 7)})

#### лучшие параметры модели 

In [11]:
best_params_tree = cls_tree.best_params_
cls_tree.best_score_

0.8013538239915764

#### обучение модели на лучших параметрах 

In [12]:
tree = DecisionTreeClassifier(**best_params_tree, random_state=1234)
tree.fit(X_train, y_train)


accuracy_score(y_valid, tree.predict(X_valid))

0.7713841368584758

### Ближайшие соседи 

#### подбор параметров

In [13]:
params = {
    'n_neighbors': range(3, 15)
    ,'weights': ('uniform', 'distance')
    ,'p': (1,2)
}

cls_knn = GridSearchCV(
    KNeighborsClassifier()
    ,param_grid=params
    ,cv=3
)

cls_knn.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(3, 15), 'p': (1, 2),
                         'weights': ('uniform', 'distance')})

#### лучшие параметры модели 

In [14]:
best_params_knn = cls_knn.best_params_
cls_knn.best_score_

0.7671238628637503

#### обучение модели на лучших параметрах 

In [15]:
knn = KNeighborsClassifier(**best_params_knn)
knn.fit(X_train, y_train)


accuracy_score(y_valid, knn.predict(X_valid))

0.744945567651633

### Рандомный лес

#### подбор параметров

In [16]:
params = {
    'n_estimators' : (10, 20 , 30, 40)
    ,'criterion' : ('gini', 'entropy', 'log_loss')
    ,'max_depth' : range(3, 10)
    ,'min_samples_leaf' : range(1, 7)
}

cls_forest = GridSearchCV(
    RandomForestClassifier()
    ,param_grid=params
    ,cv=3
)

cls_forest.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'criterion': ('gini', 'entropy', 'log_loss'),
                         'max_depth': range(3, 10),
                         'min_samples_leaf': range(1, 7),
                         'n_estimators': (10, 20, 30, 40)})

#### лучшие параметры модели 

In [17]:
best_params_forest = cls_forest.best_params_
cls_forest.best_score_

0.8117267352380214

#### обучение модели на лучших параметрах 

In [18]:
forest = RandomForestClassifier(**best_params_forest)
forest.fit(X_train, y_train)


accuracy_score(y_valid, forest.predict(X_valid))

0.7931570762052877

### Регрессия 

#### подбор параметров

In [19]:
params = {
    'penalty' : ('l1', 'l2', 'elasticnet', 'none')
    ,'solver' : ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')
    ,'max_iter' : range(20, 100, 10)
    ,'C' : (1e-2, 1.0, 4.0, 5.0, 7.0)
    ,'n_jobs' : ('None', -1)
}

cls_log_reg = GridSearchCV(
    LogisticRegression(random_state=1234)
    ,param_grid=params
    ,cv=3
)

cls_log_reg.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=1234),
             param_grid={'C': (0.01, 1.0, 4.0, 5.0, 7.0),
                         'max_iter': range(20, 100, 10), 'n_jobs': ('None', -1),
                         'penalty': ('l1', 'l2', 'elasticnet', 'none'),
                         'solver': ('newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga')})

#### лучшие параметры модели

In [20]:
best_params_log_reg = cls_log_reg.best_params_
cls_log_reg.best_score_

0.7551844369187141

#### обучение модели на лучших параметрах 

In [21]:
regresion = LogisticRegression(**best_params_log_reg, random_state=1234)

regresion.fit(X_train, y_train)


accuracy_score(y_valid, regresion.predict(X_valid))

0.7325038880248833

### Вывод

- Я взял 4 варианта моделей:
    - Решающие деревья 
    - Рандомный лес
    - Ближайшие соседи
    - Логистическую регрессию

- Так же к каждой модели применял разные гиперпараметры через **GreadSerchCV**, можно было через циклы , но пришлось бы делать вложенные циклы порядка >= 3 , так как я передавал от 3 гиперпараметров в каждую модель

- После на лучших гиперпараметрах учил модель и делал предсказание на валидационной выборке для каждой модели

- Лучшей моделью для данной задачи оказалась **RandomForestClassifier** 
    - на обучающей выборки я получил точнось ~ 82%
    - на валидационной ~ 80%
- Не сильно отстает и решиющие деревья

- В итоге я выбрал для предсказания на тестовой выборке **рандомный лес**

## Проверьте модель на тестовой выборке

In [22]:
best_predict = forest.predict(X_test)

print(f'точность на тестовой выборке {accuracy_score(y_test, best_predict)}')

print(f'полнота на тестовой выборке {recall_score(y_test, best_predict)}')

print(f'точность на тестовой выборке {precision_score(y_test, best_predict)}')

best_params_forest

точность на тестовой выборке 0.8289269051321928
полнота на тестовой выборке 0.5163043478260869
точность на тестовой выборке 0.8189655172413793


{'criterion': 'gini',
 'max_depth': 9,
 'min_samples_leaf': 4,
 'n_estimators': 40}

- После предсказания на тестовых данных получили достатосно не плохую точность 
- средние recall и precision
- так же вывел при каких параметрах происходило обучение

## (бонус) Проверьте модели на адекватность

In [37]:
from sklearn.dummy import DummyClassifier

clf_dummy = DummyClassifier(random_state=42)

clf_dummy.fit(X_train, y_train)
y_pred = clf_dummy.predict(X_test)
f'''dummy_accuracy {accuracy_score(y_test ,y_pred)}, best_accuracy {accuracy_score(y_test, best_predict)}, разница {accuracy_score(y_test, best_predict) - accuracy_score(y_test ,y_pred) } '''

'dummy_accuracy 0.713841368584759, best_accuracy 0.8289269051321928, разница 0.11508553654743381 '

получается модель рандомного леса переобучена на 0.115